In [ ]:
from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline
from quantopian.pipeline import factors, filters, classifiers
from quantopian.pipeline.classifiers import Classifier
from quantopian.pipeline.factors import CustomFactor, Returns, SimpleMovingAverage, DailyReturns, BollingerBands
from quantopian.pipeline.filters import StaticAssets, Q500US, Q1500US, Q3000US, QTradableStocksUS
from quantopian.pipeline.classifiers.fundamentals import Sector  
from quantopian.pipeline.data.builtin import USEquityPricing

from quantopian.pipeline.data import Fundamentals

import alphalens
import datetime
import numpy as np
import pandas as pd



In [ ]:
start_date  = '2015-01-01'
end_date    = '2018-11-20'

universe = QTradableStocksUS()
sma =SimpleMovingAverage( inputs=[USEquityPricing.close],   window_length=20) 
bband = BollingerBands( inputs=[USEquityPricing.close],   window_length=20, k=2) 

sector = Sector(mask=universe)
bb_u = bband.upper
bb_l = bband.lower
zscore_u = bb_u.zscore()
zscore_l = bb_l.zscore()

pipe = Pipeline()
pipe.add(sma, "sma")
pipe.add(sector, "sector")  
#pipe.add(zscore, "zscore")
pipe.add(bb_u, "bb_u")
pipe.add(bb_l, "bb_l")
pipe.add(zscore_u, "zscore_u")
pipe.add(zscore_l, "zscore_l")

pipe.add(bband.upper, "bband_up")
pipe.add(bband.lower, "bband_low")
pipe.add(USEquityPricing.close.latest, "close")
pipe.set_screen(universe)
factor = run_pipeline(pipe, start_date=start_date, end_date=end_date, chunksize=250)


In [ ]:
factor.loc["2015-01-02"].sort_values(by="zsc")

In [ ]:
df = factor.xs(symbols('aapl'), level=1) 
df[[ "bb_l", "bb_u", "close"]].plot(secondary_y=["zscore_l","zscore_u", ]) #"zscore_l","zscore_u",


In [ ]:
df = factor.xs(symbols('GE'), level=1) 
df[[ "bb_l", "bb_u", "close"]].plot(secondary_y=["zscore_l","zscore_u", ])#"zscore_l","zscore_u",


In [ ]:
df = factor.xs(symbols('KO'), level=1) 
df[[ "bb_l", "bb_u", "zscore_l","zscore_u","close"]].plot(secondary_y=["zscore_l","zscore_u", ])


In [ ]:
aapl.head()

In [ ]:
z = factor["zscore_u"].reset_index().pivot("level_0", "level_1", "bb_l")
z

In [ ]:
z.head().rank(axis=1,numeric_only=True)


In [ ]:
sid_universe = set( factor.index.levels[1].unique() )

prices = get_pricing(sid_universe, start_date=start_date, end_date=end_date, fields='open_price')


In [ ]:
all_sectors = dict(Sector.SECTOR_NAMES)
all_sectors[-1] = "Unknown"
factor_data = alphalens.utils.get_clean_factor_and_forward_returns(factor=factor["factor"],
                                                                   prices=prices,
                                                                   groupby=factor["sector"],
                                                                   quantiles=None,
                                                                   bins=1,
                                                                   periods=(1, 11, 22),
                                                                   filter_zscore=None,
                                                                   groupby_labels=all_sectors)



In [ ]:
factor_data

In [ ]:
from quantopian.pipeline.filters import QTradableStocksUS
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 


class IndustryMeanPE(CustomFactor):  
    inputs = [Fundamentals.pe_ratio, Fundamentals.morningstar_industry_group_code]  
    window_length = 252

    def compute(self, today, assets, out, pe, industry_codes):  
        df = pd.DataFrame(index=assets, data={"pe_ratio": pe[-1], "industry_codes": industry_codes[-1]}) 

        out[:] = df.groupby("industry_codes").transform(np.mean).values.flatten()  

        

start_date  = '2015-01-01'
end_date    = '2018-11-20'

universe = QTradableStocksUS()
pipe = Pipeline()

pe_ratio = Fundamentals.pe_ratio.latest
filterd_pe_ratio =  (pe_ratio < 40) & (pe_ratio > 0) 

pipe.add(USEquityPricing.close.latest, "close")
pipe.add(pe_ratio, "pe_ratio")
pipe.add(filterd_pe_ratio, "filterd_pe_ratio")

pipe.set_screen(universe)

results = run_pipeline(pipe, start_date=start_date, end_date=end_date, )

results.head()

In [ ]:
"""
https://america-kabu.com/2017/10/08/us-stock-2017-4q/#toc4 の人が言うには，過去25年の平均PERは16倍
では，取得出来たpe_ratioを見てみましょう．
"""

# subplot(行数、列数、プロット番号)
fig = plt.figure()

ax1 = fig.add_subplot(1, 2, 1)
ax1.hist(results["pe_ratio"].dropna(), bins=100)
ax1.set_title("pe_ratio")

ax2 = fig.add_subplot(1, 2, 2)
ax2.hist(results[results["filterd_pe_ratio"]]["pe_ratio"], bins=100)
ax2.set_title("pe_ratio, 0~40")


"""
0-40拡大図を見ると，確かに頂点が16くらいと言ってもいい感じですね．
．
"""

In [ ]:
"""
そもそもPERってどのような動きをするのか確認しておきましょう
"""
pe_ratio =  results["pe_ratio"].reset_index().pivot("level_0", "level_1", "pe_ratio")
pe_ratio[[symbols("AAPL"), symbols("V"), symbols("PM")]].plot()
pe_ratio[[symbols("AAPL"), symbols("V"), symbols("PM")]].pct_change().cumsum().plot()

In [ ]:
"""
生のPERではうまく行かない

時系列
業種

もしその会社がどんどんだめになってるのでは、語ってくれない
あくまでスナップショットなので、
価格を見る（ボリバンとかRSI）よりも語りが少ない

"""

In [ ]:
"""
PERとCloseを並べて確認してみます
"""
df = results.xs(symbols('AAPL'), level=1) 
df[["close", "pe_ratio"]].plot(secondary_y="pe_ratio", title="Apple")

df = results.xs(symbols('V'), level=1) 
df[["close", "pe_ratio"]].plot(secondary_y="pe_ratio", title="VISA")

df = results.xs(symbols('PM'), level=1) 
df[["close", "pe_ratio"]].plot(secondary_y="pe_ratio", title="Philip Morris")



In [ ]:
"""
こうやって見ると，銘柄でPERの水準は全然違います．
同業種ではどうでしょう？

"""

pe_ratio[[symbols("MO"), symbols("PM")]].plot()
pe_ratio[[symbols("MO"), symbols("PM")]].pct_change().cumsum().plot()

df = results.xs(symbols('PM'), level=1) 
df[["close", "pe_ratio"]].plot(secondary_y="pe_ratio", title="Philip Morris")

df = results.xs(symbols('MO'), level=1) 
df[["close", "pe_ratio"]].plot(secondary_y="pe_ratio", title="Altria")



In [ ]:
#results["per"].replace([np.inf, -np.inf], np.nan).dropna().sort_values().describe()
# df = results.xs(symbols('TEN'), level=1) 
# df[["close", "per"]].plot(secondary_y="per")
# results[(results["per"] >= 12) & (results["per"] <= 20)]

norm_per = results["norm_per"].reset_index().pivot("level_0", "level_1", "norm_per")
norm_eps = results["norm_eps"].reset_index().pivot("level_0", "level_1", "norm_eps")
pe_ratio =  results["pe_ratio"].reset_index().pivot("level_0", "level_1", "pe_ratio")
per_norm = results["per_norm"].reset_index().pivot("level_0", "level_1", "per_norm")


In [ ]:
#norm_eps.replace([np.inf, -np.inf], np.nan).fillna("ffill").describe().loc["50%"].hist(bins=100)

norm_per_50p = norm_per.replace([np.inf, -np.inf], np.nan).fillna("ffill").describe().loc["50%"]
#norm_per_50p.hist(bins=100)
norm_per_50p[norm_per_50p.between(-100,100)].hist(bins=100)


In [ ]:
norm_per_50p = pe_ratio.replace([np.inf, -np.inf], np.nan).fillna("ffill").describe().loc["50%"]
#norm_per_50p.hist(bins=100)
norm_per_50p[norm_per_50p.between(-100,100)].hist(bins=100)

In [ ]:
norm_per_50p = per_norm.replace([np.inf, -np.inf], np.nan).fillna("ffill").describe().loc["50%"]
#norm_per_50p.hist(bins=100)
norm_per_50p[norm_per_50p.between(-2,2)].hist(bins=100)
